In [8]:
import pandas as pd

crsp = pd.read_pickle("Data/crsp.pkl")
price = pd.read_pickle('Data/price.pkl')
dividends = pd.read_pickle('Data/dividends.pkl')
bookvalue = pd.read_pickle('Data/bookvalue.pkl')
shrout = pd.read_pickle('Data/shrout.pkl')
bm_dec = pd.read_pickle('Data/bm_dec.pkl')

In [13]:
bm_dec.index = pd.to_datetime(bm_dec.index)

In [16]:
"""
    I need a dataframe which comprises returns of each position in the portfolio, every year

    Challenge is now to get structure right. all stocks of portfolio for every year in one df‚

"""
price_minus_long_monthly = []
price_minus_short_monthly = []

dividends_m_long_monthly = []
dividends_m_short_monthly = []

price_long_monthly = []
price_short_monthly = []

bottom_permnos = []
top_permnos = []

for year in sorted(set(date.year for date in bm_dec.index)):
     
    date = pd.Timestamp(f"{year}-06")
    date_dec = pd.Timestamp(f"{year-1}-12")
    if date_dec not in bm_dec.index:
        continue
    if year == 2018:
        continue
    # Timeframe: July-June
    month_range = pd.date_range(start=date + pd.DateOffset(months=1), periods=12, freq="ME")
    month_range = month_range.to_period('M')

    target_date = f"{year}-06" 
    top_date = pd.to_datetime(target_date)
    crsp['date'] = pd.to_datetime(crsp['date'])

    #Top Permnos
    if len(top_permnos) == 0:
        print("This is year: ", year) #Dummy
    else:
        for m in month_range:
            #Computations for monthly returns
            date = m.to_timestamp()
            shrout_filtered_m_long_monthly = crsp[(crsp['date'] == date) & (crsp['permno'].isin(top_permnos))]
            shrout_series_m_long_monthly = shrout_filtered_m_long_monthly.set_index('permno')['adj_shrout']
            top_m_prc_monthly = price.loc[date, price.columns.intersection(top_permnos)].dropna()
            sharesoutstanding_long_m_prc = shrout_series_m_long_monthly.reindex(top_m_prc_monthly.index) *1000
            long_m_prc_monthly = top_m_prc_monthly * sharesoutstanding_long_m_prc
            for permno in long_m_prc_monthly:
                price_minus_long_monthly.append({'date': date, 'avg_value': long_m_prc_monthly})

            #Dividends
            top_m_div = dividends.loc[date, dividends.columns.intersection(top_permnos)].dropna()
            sharesoutstanding_long_m_div = shrout_series_m_long_monthly.reindex(top_m_div.index) *1000
            long_m_div = top_m_div * sharesoutstanding_long_m_div
            dividends_m_long_monthly.append({"date" : date, "avg_value" :long_m_div.sum()})

        #dividends_m_long_ts.extend(dividends_m_long_monthly)


    #Bottom Permnos
    if len(bottom_permnos) == 0:
        print("This is year (bottom): ", year) #Dummy
    else:

        for m in month_range:
            #Computations for monthly returns
            date = m.to_timestamp()
            shrout_filtered_m_short_monthly = crsp[(crsp['date'] == date) & (crsp['permno'].isin(bottom_permnos))]
            shrout_series_m_short_monthly = shrout_filtered_m_short_monthly.set_index('permno')['adj_shrout']
            bottom_m_prc_monthly = price.loc[date, price.columns.intersection(bottom_permnos)].dropna()
            sharesoutstanding_short_m_prc_monthly = shrout_series_m_short_monthly.reindex(bottom_m_prc_monthly.index) *1000
            short_m_prc_monthly = bottom_m_prc_monthly * sharesoutstanding_short_m_prc_monthly
            price_minus_short_monthly.append({'date': date, 'avg_value': short_m_prc_monthly.sum()})

            #Dividends
            bottom_m_div = dividends.loc[date, dividends.columns.intersection(bottom_permnos)].dropna()
            sharesoutstanding_short_m_div = shrout_series_m_short_monthly.reindex(bottom_m_div.index) *1000
            short_m_div = bottom_m_div * sharesoutstanding_short_m_div
            dividends_m_short_monthly.append({"date" : date, "avg_value" :short_m_div.sum()})

    """"
        Stock Selection:

        All Stocks in the available Dataset are being sorted in a descending order by Book-to-market ratio.

        "Clean Row" accesses the DataFrame bm_dec, which contains the book-to-market ratios in wide-format. 
        Therefore the length of a row is equal to the number of available stocks. Since the only condition for a stock
        to be picked is that it is listed at the day of the portfolio selection, there is no look-ahead bias.

        The top 30% of stocks are being selected for the long-positions, the bottom 30% for the short-positions, respectively.


    """
    #Only stocks which contain values for June of a year get selected. Later months are not considered -> no look-ahead bias
    clean_row = bm_dec.loc[date_dec].dropna().sort_values(ascending=False) 
    n = len(clean_row)
    if n == 0:
        continue

    k = int(n * 0.3)
    bottom_permnos = clean_row.iloc[:k].index.tolist() #Selects permnos with the lower B/M-ratios
    top_permnos = clean_row.iloc[-k:].index.tolist() #Selects permnos with the higher B/M-ratios

    for m in month_range:
        #Computations for monthly returns
        date = m.to_timestamp()
        #Short-Positions
        shrout_filtered_short_monthly = crsp[(crsp['date'] == date) & (crsp['permno'].isin(bottom_permnos))]
        shrout_series_short_monthly = shrout_filtered_short_monthly.set_index('permno')['adj_shrout']
        bottom_prc_monthly = price.loc[date, price.columns.intersection(bottom_permnos)].dropna()
        sharesoutstanding_short_prc_monthly = shrout_series_short_monthly.reindex(bottom_prc_monthly.index) *1000
        short_prc_monthly = bottom_prc_monthly * sharesoutstanding_short_prc_monthly
        if short_prc_monthly.notna().any():
            price_short_monthly.append({'date': date, 'avg_value': short_prc_monthly.sum()})
        else:
            print(f"No valid prices for {date} — skipping.")
        
        #Long-Positions
        shrout_filtered_long_monthly = crsp[(crsp['date'] == date) & (crsp['permno'].isin(top_permnos))]
        shrout_series_long_monthly = shrout_filtered_long_monthly.set_index('permno')['adj_shrout']
        top_prc_monthly = price.loc[date, price.columns.intersection(top_permnos)].dropna()
        sharesoutstanding_long_prc_monthly = shrout_series_long_monthly.reindex(top_prc_monthly.index) *1000
        long_prc_monthly = top_prc_monthly * sharesoutstanding_long_prc_monthly
        if long_prc_monthly.notna().any():
            price_long_monthly.append({'date': date, 'avg_value': long_prc_monthly.sum()})
        else:
            print(f"No valid prices for {date} — skipping.")


This is year:  1961
This is year (bottom):  1961


In [24]:
nested_list = price_minus_long_monthly

# Convert each dictionary to a DataFrame
df_list = []

for entry in nested_list:
    date = entry['date']
    series = entry['avg_value']  # a pandas Series with permno as index
    
    # Convert to DataFrame
    df = series.reset_index()
    df.columns = ['permno', 'avg_value']
    df['date'] = date
    
    df_list.append(df)

# Now concatenate the list of DataFrames
long_df = pd.concat(df_list, ignore_index=True)

# Set multi-index
long_df.set_index(['date', 'permno'], inplace=True)
long_df

KeyboardInterrupt: 